In [1]:
%pylab inline
import numpy as np
import pandas as pd
data = []
datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_2014.01.30.csv'
data = pd.read_csv(datapath)
#data_CNF14 = data[data.Series == 'CNF14']
#data = data_CNF14
data.head()

Populating the interactive namespace from numpy and matplotlib


,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity
0,CNG14,1,2014-01-30D04:27:08.581903900,BFC92EA46AE44FA4,1,700000,1,,A,682000,1
1,CNG14,2,2014-01-30D04:27:08.581903900,0F50C53613C0F881,1,682000,1,,A,682000,1
2,CNG14,3,2014-01-30D04:27:08.581903900,6EA6E34859783E78,2,691000,10,,A,682000,1
3,CNG14,4,2014-01-30D04:27:08.581903900,48714F4C6C254E45,1,665000,50,,B,665000,150
4,CNG14,5,2014-01-30D04:27:08.581903900,C0F94F4C3F5B217B,1,665000,50,,B,665000,150


In [2]:
data_CNF = data[data.Series == data.Series[0]]
data = data_CNF

In [3]:
data.head()

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity
0,CNG14,1,2014-01-30D04:27:08.581903900,BFC92EA46AE44FA4,1,700000,1,,A,682000,1
1,CNG14,2,2014-01-30D04:27:08.581903900,0F50C53613C0F881,1,682000,1,,A,682000,1
2,CNG14,3,2014-01-30D04:27:08.581903900,6EA6E34859783E78,2,691000,10,,A,682000,1
3,CNG14,4,2014-01-30D04:27:08.581903900,48714F4C6C254E45,1,665000,50,,B,665000,150
4,CNG14,5,2014-01-30D04:27:08.581903900,C0F94F4C3F5B217B,1,665000,50,,B,665000,150


In [14]:
import time 
def insert(order_book_data,data_to_insert,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position:]
    return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)

start  =  time.time()

order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
y = data[data['TimeStamp'].str.contains(timestamp)]
bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference"]]) # bid
ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))

order_book_bid.append(0)
order_book_ask.append(0)
#a = []
#b = []

for i in range(len(y),3200,1):
    print '------------------------------------------------------------------------------'
    print i
    #print data.BidOrAsk[i] == 'A'
    if data.BidOrAsk[i] == 'A':

        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            
            if data.TimeStamp[i] == x1[temp_ask]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                if position_ == 0 and len(order_book_ask[temp_ask]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:#or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error2 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'   
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:# or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error3 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                
                        continue
                elif position_ == 0 and len(order_book_ask[temp_ask]) == 1:
                    print 'position and Best Price is True' 
                
            elif data.TimeStamp[i] != x1[temp_ask]:
                ###a.append(x1[temp_ask])
                #print order_book_ask[temp_ask]
                #print temp_ask
                if temp_ask == 0:
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    #print order_book_ask[temp_ask]
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    #print order_book_ask[temp_ask]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    #print 'position = %d'%(position_)
                    #print 'length-1 = %d'%(len(order_book_ask[temp_ask])-1)
                    #print order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                    #print order_book_ask[temp_ask][position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                    #print order_book_ask[temp_ask]['Price'][0:1].iloc[0]# != data['BestPrice'][i]
                    if position_ == 0 and len(order_book_ask[temp_ask]) > 1:
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error11 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                    
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error22 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        
                        if order_book_ask[temp_ask][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:# or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error33 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            continue
                        else:
                            print 'position and Best Price is True'
                            continue
                    elif position_ == 0 and len(order_book_ask[temp_ask]) == 1:
                        print 'position and Best Price is True'
                        continue
                        
                    #print order_book_ask[temp_ask]                        
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                    order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1

                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                        
                    if position_ == 0 and len(order_book_ask[temp_ask]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                        
                            continue
                    elif position_ == 0 and len(order_book_ask[temp_ask]) == 1:
                        print 'position and Best Price is True'
                        continue                            
                            
                            
                            
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x1[temp_ask]: 
                #print temp_ask
                #print order_book_ask[temp_ask]
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]

                if position_ == 0 and len(order_book_ask[temp_ask]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error1111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error2222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error3333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue                        
                    else:
                        print 'position and Best Price is True'                        
                        continue
                elif position_ == 0 and len(order_book_ask[temp_ask]) == 1:
                    print 'position and Best Price is True'
                    continue

            elif data.TimeStamp[i] != x1[temp_ask]:
                ###a.append(x1[temp_ask])
                #print temp_ask
                #print order_book_ask[temp_ask]
                order_book_ask.append(order_book_ask[temp_ask])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1
                #print 'temp_ask2 = %d'%(temp_ask)
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]
                
                if position_ == 0 and len(order_book_bid[temp_bid]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error11111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error22222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error33333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue                        
                    else:
                        print 'position and Best Price is True'                        
                        continue 
                elif position_ == 0 and len(order_book_ask[temp_bid]) == 1:
                    print 'position and Best Price is True'
                    continue                          

    elif data.BidOrAsk[i] == 'B':
        #print i
        #print int(data[['QuantityDifference']][i:i+1].values) > 0
        
        if int(data[['QuantityDifference']][i:i+1].values) > 0: #and len(order_book_bid[0]) > 0:
            print data.TimeStamp[i] == x2[temp_bid]
            
            if data.TimeStamp[i] == x2[temp_bid]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                #print position_
                print order_book_bid[temp_bid]
                if position_ == 0 and len(order_book_bid[temp_bid]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error111111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                        
                elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error222222 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'   
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[temp_bid]) > 1:
                        
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:# or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error333333 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                
                        continue
                elif position_ == 0 and len(order_book_bid[temp_bid]) == 1:
                    print 'position and Best Price is True'
                    continue
                
            elif data.TimeStamp[i] != x2[temp_bid]:
                ###b.append(x2[temp_bid])
                #print 'temp bid = %d'%(temp_bid)
                #print order_book_bid[temp_bid]
                if temp_bid == 0:
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    #print order_book_bid[temp_bid]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    #print order_book_bid[temp_bid]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    #print position_
                    #print len(order_book_bid[temp_bid]) - 1
                    
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0]
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    if position_ == 0  and len(order_book_bid[temp_bid]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1111111 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                            
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2222222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3333333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            continue
                        else:
                            print 'position and Best Price is True'
                            continue
                    elif position_ == 0 and len(order_book_bid[temp_bid]) == 1:
                            print 'position and Best Price is True'
                            continue
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    #print position_    
                    #print len(order_book_bid[temp_bid])-1      
                    if position_ == 0 and len(order_book_bid[temp_bid]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error11111111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error22222222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error33333333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                        
                            continue
                    elif position_ == 0 and len(order_book_bid[temp_bid]) == 1:
                        print 'position and Best Price is True'
                        continue
                        
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x2[temp_bid]: 
                #print temp_bid
                #print order_book_bid[temp_bid]
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]

                if position_ == 0 and len(order_book_bid[temp_bid]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:# or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error19 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                        
                elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error29 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if position_ > 0 and order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error39 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue
                    else:
                        print 'position and Best Price is True'                        
                        continue
                elif position_ == 0 and len(order_book_bid[temp_bid]) == 1:
                    print 'position and Best Price is True'
                    continue

            elif data.TimeStamp[i] != x2[temp_bid]:
                ###b.append(x2[temp_bid])
                #print temp_bid
                #print order_book_bid[temp_bid]
                order_book_bid.append(order_book_bid[temp_bid])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                #print position_
                #print len(order_book_ask[temp_ask])-1
                #print 'temp_bid2 = %d'%(temp_bid)
                
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]
                
                if position_ == 0 and len(order_book_bid[temp_bid]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error110 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
    
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error220 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if position_ > 0 and order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error330 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue
                    else:
                        print 'position and Best Price is True'                        
                        continue    
                elif position_ == 0 and len(order_book_bid[temp_bid]) == 1:
                    print 'position and Best Price is True'
                    continue                    
                                    
end = time.time()            
print "Total time = %f"%(end - start)    

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


------------------------------------------------------------------------------
13
False
position and Best Price is True
------------------------------------------------------------------------------
14
False
position and Best Price is True
------------------------------------------------------------------------------
15
False
position and Best Price is True
------------------------------------------------------------------------------
16
False
position and Best Price is True
------------------------------------------------------------------------------
17
False
position and Best Price is True
------------------------------------------------------------------------------
18
False
position and Best Price is True
------------------------------------------------------------------------------
19
------------------------------------------------------------------------------
20
position and Best Price is True
------------------------------------------------------------------------------
21
Fa

In [18]:
data[(data.Price == 675000)].head(50)

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity
34,CNG14,35,2014-01-30D08:46:00.036830600,F422E301EBC7D007,1,675000,20,,A,675000,20
51,CNG14,52,2014-01-30D08:46:00.065830600,F4A2E309E7BFCBFF,2,675000,130,,A,675000,150
66,CNG14,67,2014-01-30D08:46:15.379650600,0D89C526DDF7C2B7,1,675000,1,,B,675000,1
72,CNG14,73,2014-01-30D08:46:39.409362600,74224A0DC315A215,1,675000,1,,B,675000,2
73,CNG14,74,2014-01-30D08:46:44.307302600,8D09C52EC99DAE5D,3,675000,2,,B,675000,4
74,CNG14,75,2014-01-30D08:46:46.771266600,FA6ACA4D2D540C54,1,675000,1,,B,675000,5
75,CNG14,76,2014-01-30D08:46:50.157230600,8D09C52EC99DAE5D,4,675000,-2,,B,675000,3
76,CNG14,77,2014-01-30D08:46:50.322230600,0D89C526DDF7C2B7,3,675000,-1,,B,675000,2
77,CNG14,78,2014-01-30D08:46:55.298170600,70BE6E1D4F742E54,3,675000,1,,B,675000,3
78,CNG14,79,2014-01-30D08:47:06.746031400,F4AA4A0D5D3D3C3D,4,675000,1,,B,675000,4


In [12]:
order_book_bid[len(order_book_bid)-1]

,Price,OrderNumber,QuantityDifference
0,674500,FA6ACA4D2D540C54,1
1,675000,74224A0DC315A215,1
2,675000,70BE6E1D4F742E54,1
3,675000,F4AA4A0D5D3D3C3D,1
4,675000,F2E2464DF4BAD3BA,1
5,675000,F6FA660D46B32593,1
6,675000,F036EE1D2D9F0C7F,1
7,675000,D4EAE24552613141,1
8,675000,F26AC645BC7C9B7C,1
9,675000,F67AE60503F8E2D9,1


In [30]:
data.head(790)

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity
0,CNG14,1,2014-01-30D04:27:08.581903900,BFC92EA46AE44FA4,1,700000,1,,A,682000,1
1,CNG14,2,2014-01-30D04:27:08.581903900,0F50C53613C0F881,1,682000,1,,A,682000,1
2,CNG14,3,2014-01-30D04:27:08.581903900,6EA6E34859783E78,2,691000,10,,A,682000,1
3,CNG14,4,2014-01-30D04:27:08.581903900,48714F4C6C254E45,1,665000,50,,B,665000,150
4,CNG14,5,2014-01-30D04:27:08.581903900,C0F94F4C3F5B217B,1,665000,50,,B,665000,150
5,CNG14,6,2014-01-30D04:27:08.581903900,6CB163000307E808,3,660000,100,,B,665000,150
6,CNG14,7,2014-01-30D04:27:08.581903900,662E6B4847A22CA2,3,699000,10,,A,682000,1
7,CNG14,8,2014-01-30D04:27:08.581903900,2BD5C16638561AB6,5,717500,10,,A,682000,1
8,CNG14,9,2014-01-30D04:27:08.581903900,D8F8E619640F43AF,2,683500,1,,A,682000,1
9,CNG14,10,2014-01-30D04:27:08.581903900,C4F76F586C344E54,3,689000,100,,A,682000,1
